<a href="https://colab.research.google.com/github/medrante/spectral_project/blob/main/spectral_project_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spectral Project (Thesis)

## Outline
- Install libraries
- Set variables
- Create telescope aperture
- Helper functions
  - Create first / main wavefront
  - Propagate main wavefront
  - Create other wavefronts
  - Propagate them
  - Generate power spectrum
- Save all the images

- All the resnets

## Install Libraries

If you are running this notebook on Colab, you can run the following cell to install the libraries we'll use.

If you are running this notebook on your own computer, you might have to install these libraries yourself.  See the instructions in the preface.

In [2]:
# If we're running on Colab, install libraries

# TODO: When Colab can install gala, switch from astro-gala

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 1.4 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [9]:
# https://github.com/medrante/spectral_project/blob/main/color_magnitude_table.csv

import pandas as pd
url = 'https://raw.githubusercontent.com/medrante/spectral_project/main/color_magnitude_table.csv?token=GHSAT0AAAAAACQFH3UZCZPUKS67R56EV4BEZQGBZJQ'
df = pd.read_csv(url,index_col=0)

df

,mag_s_v,color_s_v,mag_u_v,color_u_v,mag_b_v,color_b_v,mag_V,color_V,mag_v_w,color_v_w,mag_v_x,color_v_x,mag_v_p,color_v_p,mag_v_z,color_v_z
Star,,,,,,,,,,,,,,,,
SA 94_06,8,-0.356,6,-0.374,5,-0.179,7,8.725,4,-0.090,6,-0.110,5,-0.149,6,-0.162
SA 94_07,8,2.466,5,2.128,6,0.801,8,8.855,4,0.491,6,0.831,6,0.945,7,1.041
SA 94_08,14,0.964,6,0.840,6,0.338,8,9.017,5,0.212,5,0.375,5,0.428,9,0.485
SA 94_10,8,0.140,5,0.154,5,0.064,7,10.083,4,0.055,5,0.106,6,0.100,10,0.110
SA 94_12,7,-0.476,6,-0.490,6,-0.280,8,10.488,6,-0.129,5,-0.187,6,-0.215,6,-0.247
SA_71 _07,9,1.599,8,1.416,8,0.581,8,10.305,4,0.398,8,0.673,6,0.771,7,0.885
SA_71_11,10,0.104,7,0.088,6,0.037,7,10.779,6,0.047,7,0.083,6,0.099,9,0.097
SA_71_14,9,1.317,6,1.140,6,0.471,7,11.264,5,0.301,9,0.509,11,0.585,8,0.650
SA_71_17,8,-0.200,6,-0.202,5,-0.079,7,11.757,4,0.015,6,0.043,7,0.060,9,0.066


## Helper Functions

### Create wavefront

### Propagate wavefront

## Generation all the images